In [1]:
# Librerias:
import requests as req
import lxml.html as html
# Para la casi ultima clase
import os
import datetime


In [2]:
HOME_URL = 'https://www.larepublica.co/'
XPATH_LINK_TO_ARTICLE = '//a[contains(@class,"kicker")]/@href'
XPATH_TITLE = '//div[@class = "wrap-info"]/h1/text()'
XPATH_SUMMARY = '//div[@class = "lead"]/p/text()'
XPATH_BODY ='//div[@class="html-content"]//text()'

In [3]:
def get_title(link):
    #separamos por "/" y nos quedamos con el ultimo que elemento 
    url = link.split('/')[-1]
    #separamos por "-" y eliminamos el ultimo elemento
    title_list=url.split('-')[:-1]
    #Unimos lo anterior
    title = " ".join(title_list)
    return(title)

def parsed_notice(link, today):
    try:
        response = req.get(link)
        if response.status_code == 200:
            #Traer el doc html:
            notice = response.content.decode('utf-8')
            parsed = html.fromstring(notice)

            try:
                # Devuelde la lista pero solo queremos el titulo
                title = get_title(link)
                summary = parsed.xpath(XPATH_SUMMARY)[0]
                body = parsed.xpath(XPATH_BODY)
            except IndexError:
                return
            #Guardar en archivo:
            # Ver función de f ""
            with open(f'{today}/{title}.txt', 'w', encoding='utf-8') as f:
                f.write(title)
                f.write('\n\n')
                f.write(summary)
                f.write('\n\n')
                for p in body:
                    f.write(p)
                    f.write('\n')

        else:
            raise ValueError(f'Error: {response.status_code}')
    except ValueError as ve:
        print(ve)

In [4]:
def pars_home():
    try:
        response = req.get(HOME_URL)
        if response.status_code == 200:
            # El metodo content trae todo el documento html
            # El metodos decode permite no tener errores con caracteres especiales
            home = response.content.decode('utf-8')
            # Tranforma el contenido htlm para poder hacer spam
            parsed = html.fromstring(home)
            # El metodo parsed
            links_notice = parsed.xpath(XPATH_LINK_TO_ARTICLE)
            #print(links_notice)

            #Variable para crear la carpeta de almacenamiento:
            today = datetime.date.today().strftime('%d-%m-%Y')
            # Si no existe la carpeta una carpeta con la fecha
            # de hoy crearla:
            if not os.path.isdir(today):
                os.mkdir(today)

            for link in links_notice:
                parsed_notice(link, today)
        else:
            raise ValueError(f'Error: {response.status_code}')
    except ValueError as ve:
        print(ve)


In [5]:
def run():
    pars_home()

In [6]:
if __name__ == '__main__':
    run()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=5de5f16a-6fc2-4d62-b482-732e375eb9ab' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>